In [1]:
import pandas as pd
import ast
import configparser
import zipfile
import os
import textwrap

from neo4j import GraphDatabase

# Data Preparation and Neo4j Ingest

In this notebook we format the 
[Food.com Recipes and Interactions Dataset](https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv) [[1]](#1) and load it into a Neo4j graph. The work is divided into two primary sections in this notebook:

1. __[Prepare Data (Extract & Transform)](#prepare-data)__
2. __[Load Data Into Neo4j](#load-data)__

__To run this notebook please do the following:__
 - Download the training and validation sets from [Food.com Recipes and Interactions Dataset](https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv) and place into the `SOURCE_DATA_PATH` set in the below [Setup Parameters](#set-up) section.

 - Ensure the `NEO4J_IMPORT_DIR`, `NEO4j_IMPORT_PATH`, and `NEO4J_PROPERTIES_FILE` are set correctly for your environemnt in the below [Setup Parameters](#set-up) section.

__Other Prerequisites:__
- Neo4j. This notebook was tested with [Neo4j Desktop](https://neo4j.com/download-center/#desktop) and should work for other on-prem installations.
- [APOC library](https://neo4j.com/labs/apoc/4.3/installation/). 
- Notebook tested with Python=3.8

<a id="1">[1]</a>
Bodhisattwa Prasad Majumder, Shuyang Li, Jianmo Ni, and Julian McAuley. 2019. Generating Personalized Recipes from Historical User Preferences. In Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP), pages 5976–5982, Hong Kong, China. Association for Computational Linguistics.

## <a id="set-up">Setup Parameters</a>

#### Directory Settings

In [2]:
# Data directory
SOURCE_DATA_PATH = './data/'

`NEO4J_IMPORT_DIR` sets the the directory to put formatted csvs in for Neo4j load.
File locations for each OS are listed in the [Neo4j Operations Manual](https://neo4j.com/docs/operations-manual/current/configuration/file-locations). We recommend putting files in Neo4j’s `import` directory, as it keeps the environment secure. However, if you need to access files in other locations, you can find out which setting to alter in our [manual](https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/#query-load-csv-introduction).

In [3]:
NEO4J_IMPORT_DIR = '/home/luigi/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e8e4d675-a4f4-4731-aff0-a9cc0edee8c4/import'

`NEO4j_IMPORT_PATH` sets the path to use during CSV load. This will be 'file:///' by default and aligns with the local Neo4j `import` directory setup. 
See https://neo4j.com/developer/guide-import-csv/#_reading_csv_files for more details and customization

In [4]:
NEO4j_IMPORT_PATH = 'file:///'

#### Neo4j Settings
The `NEO4J_PROPERTIES_FILE` is an ini configuration file for Neo4j properties so this notebook can connect to
your Neo4j instance and load data. The ini file should be formatted as follows
```
[NEO4J]
PASSWORD=<password>
DATABASE=<database name, is 'neo4j' by default>
HOST=<host uri>

```

Set `NEO4J_PROPERTIES_FILE` to None or any non-existent location to use the below defaults
```
HOST = 'neo4j://localhost'
DATABASE = 'neo4j'
PASSWORD = 'password'
```

In [5]:
NEO4J_PROPERTIES_FILE = None # 'path/to/your/neo4j_properties.ini'

#### Impression Chunk Size
The number of rows to work with at once when reading from source behaviors/impressions files. Recommend keeping
this below 100,000 - ish unless you know your machine can handle more rows efficiently.

In [6]:
IMPRESSION_CHUNK_SIZE = 100_000

## <a id="prepare-data">Prepare Data (Extract & Transform)</a>
In this section we extract and transform the source data into formatted csv files to prepare for [loading into Neo4j](#load-data).
We will create the following csv files:

__Recipes related graph elements__
1. `recipe.csv`: Nodes representing recipes
2. `ingredients.csv`: Nodes representing ingredients
3. `with_ingredients.csv`: Relationship representing recipe having a set of ingredients
4. `created.csv`: Relationship representing user creating (posting) a recipe
 
__User and impression related graph elements__
1. `users.csv`: Nodes representing anonymized users
2. `reviewed.csv`: Relationship representing user reviewing a recipe

### Unzip Source data

In [7]:
# Extract zip file to SOURCE_DATA_PATH
with zipfile.ZipFile(SOURCE_DATA_PATH + "archive.zip","r") as zip_ref:
    zip_ref.extractall(SOURCE_DATA_PATH)

### Recipe Related Graph Elements

Let's start by uploading the raw recipes csv file as a dataframe

In [8]:
# Read RAW_recipes.csv file to raw_recipes_df dataframe
raw_recipes_df = pd.read_csv(f'{SOURCE_DATA_PATH}RAW_recipes.csv')
raw_recipes_df.head(n=2)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6


#### Recipe Nodes
For the recipe node we will basically just drop some columns from the raw_recipes_df and just keep the following columns:
1. `name`: recipe name
2. `id`: recipe id
3. `minutes`: minutes needed to prepare the recipe
4. `tags`: food.com tags for the recipe
5. `nutrition`: nutrition information (calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat)
6. `n_steps`: number of steps in the recipe
7. `steps`: text for recipe steps in order
8. `description`: user-provided description of the recipe
9. `n_ingredients`: number of ingredients needed

In [43]:
print(recipes_df['description'].isnull().sum())

4979


In [9]:
# Create recipes dataframe
recipes_df = raw_recipes_df[['name', 'id', 'minutes', 'tags', 'nutrition', 'n_steps', 'steps', 'description', 'n_ingredients']]
# Set default values
recipes_df['name'].fillna('No Name', inplace=True)
recipes_df['description'].fillna('No Description', inplace=True)
# Save as csv file to NEO4J_IMPORT_DIR
recipes_df.to_csv(f'{NEO4J_IMPORT_DIR}/recipes.csv', index = False)
# Show dataframe
recipes_df.head(n=2)

,name,id,minutes,tags,nutrition,n_steps,steps,description,n_ingredients
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,7
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,6


#### Ingredients Nodes
For the ingredients node we will create a set of ingredients iterating all recipies and create a dataframe with the following colums:
1. `id`: ingredient ID
2. `name`: ingredient name

In [10]:
# Create an empty set to store unique ingredients
ingredients_dict = dict()
id_counter = 1

# Iterate over all lists of ingredients
for ingredients_list in raw_recipes_df['ingredients']:
    # Iterate over all ingredients
    for ingredient in ast.literal_eval(ingredients_list):
        # Add ingredient to the dictionary (if not present)
        if ingredient not in ingredients_dict:
            ingredients_dict[ingredient] = id_counter
            id_counter += 1

# Create a dataframe from the ingredients dictionary
ingredients_df = pd.DataFrame(list(ingredients_dict.items()), columns=['name', 'id'])
# Save as csv file to NEO4J_IMPORT_DIR
ingredients_df.to_csv(f'{NEO4J_IMPORT_DIR}/ingredients.csv', index = False)
# Show dataframe
ingredients_df.head(n=2)

,name,id
0,winter squash,1
1,mexican seasoning,2


#### WithIngredients Relationship



For the with_ingredients relationship we will instead create a dataframe with the following columns:
1. `recipe_id`: id of the recipe
2. `ingredient_id`: name of the ingredient

In [11]:
# Initialize an empty list for 'WITH_INGREDIENTS' relation
recipe_ingr_data = []

# Iterate through raw_recipes_df recipes
for index, row in raw_recipes_df.iterrows():
    # Iterate through the recipe's ingredients list
    for ingredient in ast.literal_eval(row['ingredients']):
        # Add to relation the entry <recipe_id>, <ingredient_id>
        recipe_ingr_data.append({'recipe_id':row['id'], 'ingredient_id':ingredients_dict[ingredient]})

# Create with_ingredients dataframe from recipe_ingr_data
with_ingredients_df = pd.DataFrame(recipe_ingr_data)
# Save as csv file to NEO4J_IMPORT_DIR
with_ingredients_df.to_csv(f'{NEO4J_IMPORT_DIR}/with_ingredients.csv', index = False)
# Show dataframe
with_ingredients_df.head(n=2)

,recipe_id,ingredient_id
0,137739,1
1,137739,2


#### Created Relationships

The created relationship will include data about which user created a recipe with the following columns:
1. `recipe_id`: recipe ID
2. `user_id`: user ID of the contributor (user who created the recipe)
3. `submitted`: date recipe was submitted

In [12]:
# Create created_df dataframe
created_df = raw_recipes_df[['id', 'contributor_id', 'submitted']].drop_duplicates()
# Rename columns 
created_df = created_df.rename(columns={'id': 'recipe_id', 'contributor_id':'user_id'})
# Save as csv file to NEO4J_IMPORT_DIR
created_df.to_csv(f'{NEO4J_IMPORT_DIR}/created.csv', index = False)
# Show dataframe
created_df.head(n=2)

,recipe_id,user_id,submitted
0,137739,47892,2005-09-16
1,31490,26278,2002-06-17


### User and Impression Related Graph Elements

The reviewed relation is just given by the RAW_interactions.csv, hence we can keep it as it is with the following columns:
1. `user_id`: user_id
2. `recipe_id`: recipe_id
3. `date`: date of interaction
4. `rating`: given rating (0 to 5)
5. `review`: review text

In [13]:
# Read RAW_recipes.csv file to raw_recipes_df dataframe
reviewed_df = pd.read_csv(f'{SOURCE_DATA_PATH}RAW_interactions.csv')
# Save as csv file to NEO4J_IMPORT_DIR
reviewed_df.to_csv(f'{NEO4J_IMPORT_DIR}/reviewed.csv', index = False)
# Show dataframe
reviewed_df.head(n=2)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."


#### User Nodes

For the user nodes, we will just keep unique 'user_id' from the ratings data:

In [14]:
# Create user dataframe
user_df = reviewed_df[['user_id']].drop_duplicates().rename(columns={'user_id': 'id'})
# Save as csv file to NEO4J_IMPORT_DIR
user_df.to_csv(f'{NEO4J_IMPORT_DIR}/users.csv', index = False)
# Show dataframe
user_df.head(n=2)

,id
0,38094
1,1293707


## <a id="load-data">Load Data Into Neo4j</a>

In this section we will load the above transformed data into Neo4j. As a good rule of practice for Neo4j bulk loading,
I have found it ideal if you can follow the below pattern. Note that for this to work, each node should have some sort
of external unique id, or you must otherwise create one at ingest time.

1. __Set a uniqueness constraint__ on a property for each node label
2. __Load nodes__ using `MERGE` which is similar to an upsert pattern on the above uniqueness constraint and is very well optimized in Neo4j
3. __Load relationships__ using `MATCH` and `MERGE` to find adjacent nodes on uniqueness constraint and upsert respectively

In [15]:
## Using an ini file for credentials, otherwise providing defaults
HOST = 'bolt://localhost:7687'
DATABASE = 'neo4j'
PASSWORD = 'LU?vVtYf8])9;^z'

if NEO4J_PROPERTIES_FILE is not None and os.path.exists(NEO4J_PROPERTIES_FILE):
    config = configparser.RawConfigParser()
    config.read(NEO4J_PROPERTIES_FILE)
    HOST = config['NEO4J']['HOST']
    DATABASE = config['NEO4J']['DATABASE']
    PASSWORD = config['NEO4J']['PASSWORD']
    print('Using custom database properties')
else:
    print('Could not find database properties file, using defaults')

Could not find database properties file, using defaults


In [16]:
# helper function
def run(driver, query, params=None):
    with driver.session() as session:
        if params is not None:
            return [r for r in session.run(query, params)]
        else:
            return [r for r in session.run(query)]

In [17]:
driver = GraphDatabase.driver(HOST, auth=(DATABASE, PASSWORD))

### Set Uniqueness Constraints
First, let's clear the database

In [18]:
# Delete all previous nodes and relations
run(driver, textwrap.dedent("""\
    CALL apoc.periodic.commit(
        "MATCH (n)
        WITH n LIMIT $limit
        DETACH DELETE n 
        RETURN COUNT(*)", 
    {limit:1000})
    """))

[<Record updates=0 executions=0 runtime=0 batches=1 failedBatches=0 batchErrors={} failedCommits=0 commitErrors={} wasTerminated=False>]

### Set Uniqueness Constraints
This step is critical for performant loading with merge patterns

In [19]:
run(driver,'CREATE CONSTRAINT user_id_unique IF NOT EXISTS FOR (user:User) REQUIRE user.id  IS UNIQUE')
run(driver,'CREATE CONSTRAINT recipe_id_unique IF NOT EXISTS FOR (recipe:Recipe) REQUIRE recipe.id IS UNIQUE')
run(driver,'CREATE CONSTRAINT ingredient_id_unique IF NOT EXISTS FOR (ingredient:Ingredient) REQUIRE ingredient.id IS UNIQUE')

[]

### Load Nodes

In [20]:
# User nodes
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    CALL { WITH row  
        MERGE(user:User {id:row.id})
    } IN TRANSACTIONS OF 10000 ROWS
    """),
    params = {'file': NEO4j_IMPORT_PATH + 'users.csv'}
)

[]

In [30]:
# Recipe nodes
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    CALL { WITH row
        WITH row, COALESCE(row.description, 'No Description') AS description
        MERGE(recipe:Recipe {
            id: row.id,
            name: row.name,
            minutes: row.minutes,
            tags: row.tags,
            nutrition: row.nutrition,
            n_steps: row.n_steps,
            steps: row.steps,
            description: description,
            n_ingredients: row.n_ingredients })
    } IN TRANSACTIONS OF 10000 ROWS
    """),
    params = {'file': NEO4j_IMPORT_PATH + 'recipes.csv'}
)

ClientError: {code: Neo.ClientError.Statement.SemanticError} {message: Cannot merge the following node because of null property value for 'name': (:Recipe {name: null}) (Failure when processing file '/home/luigi/.config/Neo4j%20Desktop/Application/relate-data/dbmss/dbms-e8e4d675-a4f4-4731-aff0-a9cc0edee8c4/import/recipes.csv' on line 723., Transactions committed: 0)}